In [0]:
%run "../commons/common_functions" 

In [0]:
%run "../commons/configuration"

In [0]:
dbutils.widgets.text("env_name","DEV")
p_env_name = dbutils.widgets.get("env_name")

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import to_timestamp, concat, lit, col
from pyspark.sql.types import *


pit_stops_schema = StructType([StructField("raceId", IntegerType(), False), 
                            StructField("driverId", IntegerType(), True), 
                            StructField("stop", StringType(), True), 
                            StructField("lap", IntegerType(), True),
                            StructField("time", StringType(), True),
                            StructField("duration", StringType(), True),
                            StructField("milliseconds", IntegerType(), True)])

pit_stops_df = spark.read.json(f"{raw_mount_path}/pit_stops.json", schema=pit_stops_schema, multiLine=True)\
    .withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("driverId", "driver_id")\
    .withColumn("data_source",lit(p_env_name))

In [0]:
pit_stops_df_v2 = add_ingestion_date(pit_stops_df)

In [0]:
#pit_stops_df_v2.write.parquet(f"{processed_mount_path}/pit_stops", mode="overwrite")
pit_stops_df_v2.write.mode("overwrite").format("parquet").saveAsTable("processed.pit_stops")

In [0]:
dbutils.notebook.exit("Success")